<a href="https://colab.research.google.com/github/fmunin/RAG-Chatbot-PowerBI/blob/main/frm_RAG_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing the required libraries
!pip install openai==1.66.3 tiktoken==0.9.0 pypdf==5.4.0 langchain==0.3.20 langchain-community==0.3.19 langchain-chroma==0.2.2 langchain-openai==0.3.9 chromadb==0.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.1/602.1 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458

In [ ]:
!unzip /gdrive/MyDrive/classwork/Generative-AI/RAG-Chatbot/PowerBI.zip


In [2]:
from google.colab import drive
import os
drive.mount('/gdrive')
urlBase="/gdrive/My Drive/classwork/Generative-AI/RAG-Chatbot"
urlPowerBI=os.path.join(urlBase,"PowerBI_db")

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
#change the working directory
%cd /gdrive/My Drive/classwork/Generative-AI/RAG-Chatbot
print(f"current working directory is {os.getcwd()}")

/gdrive/My Drive/classwork/Generative-AI/RAG-Chatbot
current working directory is /gdrive/My Drive/classwork/Generative-AI/RAG-Chatbot


In [4]:
# Importing the standard Libraries
                         # For interacting with the operating system
import time                           # For measuring execution time or adding delays
from datetime import datetime         # For handling timestamps and datetime operations

# ChromaDB Vector Database
import chromadb  # Chroma: a local-first vector database for storing and querying document embeddings

# OpenAI SDK
from openai import OpenAI             # Official OpenAI Python SDK (v1.x) for interacting with models like GPT-4

# LangChain Utilities
# RecursiveCharacterTextSplitter intelligently breaks long text into smaller chunks with some overlap, preserving context.
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Loads all PDF files from a directory and extracts text from each.
from langchain_community.document_loaders import PyPDFDirectoryLoader

# Base class representing a document in LangChain; useful for downstream chaining and processing.
from langchain_core.documents import Document

# Embeddings and Vector Store
# Generates vector embeddings using OpenAI’s embedding models (e.g., `text-embedding-3-small`)
from langchain_openai import OpenAIEmbeddings

# Integration for using Chroma as the vector store within LangChain’s ecosystem
from langchain_chroma import Chroma

# Chunking Source PDF File


In [5]:
#Set up directory where PDF SOURCE files
urlpdfSource = os.path.join(urlBase,"source-docs")
print(os.path.exists(urlpdfSource))
print(urlpdfSource)

True
/gdrive/My Drive/classwork/Generative-AI/RAG-Chatbot/source-docs


In [7]:
#text splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,
    chunk_overlap=16
)

In [8]:
# Chunking the source pdf
loader = PyPDFDirectoryLoader(urlpdfSource)
chunks_PowerBI = loader.load_and_split(text_splitter=text_splitter)


In [9]:
# Inspecting content of first page
print(f"The text splitter create {len(chunks_PowerBI)} chunks of text")
print('======= Printing the first record ==========')
chunks_PowerBI[0].page_content

The text splitter create 407 chunks of text
======= Printing the first record ==========


'Introducing\nMicrosoft \nPower BI\nAlberto Ferrari and Marco Russo'

In [11]:
# initialize Chroma DB collection name
collectionName_PowerBI ="PowerBI-Documentation"

# Initialize OpenAI Embedding Model

In [12]:
#defining API key
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

In [13]:
# Instantiate the OpenAI embedding model
embedding_model = OpenAIEmbeddings(
    api_key=openai_api_key,
    base_url="https://aibe.mygreatlearning.com/openai/v1",
    model='text-embedding-3-small'
)

In [14]:
# Initialize a persistent Chroma client
chromadb_client = chromadb.PersistentClient(
    path= urlPowerBI
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


In [15]:
# Pinging the database client to check if the connection is alive
# the heartbeat method returns the current time in nanoseconds and is generally used to check if the server is alive
chromadb_client.heartbeat()

1768488112501361886

In [17]:
# Confirm database is empty
print(f"Initial Collection count = {chromadb_client.count_collections()}")

Initial Collection count = 0


In [18]:
# Instantiate a Chroma vector store to store and retrieve document embeddings
vectorstore = Chroma(
    collection_name= collectionName_PowerBI,
    collection_metadata={"hnsw:space": "cosine"},
    embedding_function=embedding_model,
    client=chromadb_client,
    persist_directory= urlPowerBI
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [19]:
# confirm collection
print(f"Collection count (After Collection Creation)= {chromadb_client.count_collections()}")

Collection count (After Collection Creation)= 1


In [20]:
# Confirm database has been populated with the collection
print("List of available collections")
chromadb_client.list_collections()

List of available collections


['PowerBI-Documentation']

In [21]:
# Batch 500 chunks to send to the API at a time, pausing execution for 30 seconds afterward
i = 0 # Initialize the starting index for the chunks

while i < len(chunks_PowerBI): # Iterate while the index is less than the total number of chunks
    vectorstore.add_documents( # Add documents to the vector store in batches of 500
        documents=chunks_PowerBI[i:i+500], # Get the current batch of 500 chunks
        ids=["text_" + str(i) for i in range(i, i+500)] # Assign unique IDs to each chunk in the batch
    )

    i += 500 # Increment the index by 500 to move to the next batch
    time.sleep(30) # Pause for 30 seconds to avoid rate limiting issues with the vector store


Saving DB to Zip File

In [ ]:
#ensure you change the notebook working directory
print(f"Current working Directory is {os.getcwd()}")
!zip -r PowerBI_db.zip PowerBi_db

Current working Directory is /gdrive/MyDrive/classwork/Generative-AI/RAG-Chatbot
updating: PowerBi_db/ (stored 0%)
updating: PowerBi_db/chroma.sqlite3 (deflated 97%)


CRUD Operations in ChromaDB

Inspecting Individual Records
---



In [28]:
#creating a Persisted vector store to access records
vectorstore_persisted = Chroma(
    collection_name= collectionName_PowerBI ,
    collection_metadata={"hnsw:space": "cosne"},
    embedding_function=embedding_model,         # Embedding model used to convert text into vectors
    client=chromadb_client,                     # Persistent Chroma client initialized earlier
    persist_directory= urlPowerBI               # directory where Chroma will persist its data
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [23]:
#Define chroma collection
PowerBI_Collection= chromadb_client.get_collection(collectionName_PowerBI)

In [24]:
# how many records are in collection??

print(f"The current PowerBI Chroma Collection has {PowerBI_Collection.count()} records")

The current PowerBI Chroma Collection has 407 records


In [27]:
#Inspect first 2 records with .peek method
PowerBI_Collection.peek(2)

{'ids': ['text_0', 'text_1'],
 'embeddings': array([[-0.0141233 , -0.03104495,  0.0387464 , ..., -0.00077508,
          0.00032513,  0.01716083],
        [ 0.0097238 ,  0.00484162,  0.02150327, ..., -0.019948  ,
         -0.01871731,  0.01614773]]),
 'documents': ['Introducing\nMicrosoft \nPower BI\nAlberto Ferrari and Marco Russo',
  'PUBLISHED BY \nMicrosoft Press \nA division of Microsoft Corporation \nOne Microsoft Way \nRedmond, Washington 98052-6399 \nCopyright © 2016 by Microsoft Corporation \nAll rights reserved. No part of the contents of \nthis book may be reproduced or transmitted in \nany form or by any means without the written \npermission of the publisher. \nISBN: 978-1-5093-0228-4 \nMicrosoft Press books are available through \nbooksellers and distributors worldwide. If you \nneed support related to this book, email \nMicrosoft Press Support at \nmspinput@microsoft.com. Please tell us what \nyou think of this book at http://aka.ms/tellpress. \nThis book is provided “as-

Observations

Retrieving Records based on user query

In [29]:
# Create retriever interface
retriever = vectorstore_persisted.as_retriever(
    search_type = 'similarity',             # Use the default method based on semantic similarity
    search_kwargs = {'k': 5}                # Retrieve top 5 most similar chunks
)



In [30]:
#Define sample query
user_query = "What is Power BI"

In [31]:
#Perform similarity search using .invoke method
retriever.invoke(user_query)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


[Document(id='text_23', metadata={'author': 'Joan', 'creationdate': '2016-06-13T10:18:21-04:00', 'creator': 'Adobe Acrobat Pro 10.1.16', 'moddate': '2016-06-13T21:13:38-04:00', 'page': 23, 'page_label': '24', 'producer': 'Adobe Acrobat Pro 10.1.16', 'source': '/gdrive/My Drive/classwork/Generative-AI/RAG-Chatbot/source-docs/Introducing_Power_BI.pdf', 'title': '', 'total_pages': 407}, page_content='6 C H A P T E R  1  |  Introducing Power BI \n \nFortunately, David heard about an interesting \ntool called Power BI that Microsoft created in \n2015 that might be helpful toward creating a \ncollaborative environment in which any \nstakeholder of the budgeting process can share \nhis findings with others, working together on the \ngoal. But, at this point, the name and maybe a \nmarketing video is all that David knows about \nPower BI. \nDriven by curiosity, he navigates to \nwww.powerbi.com\n and starts down his learning \npath. Figure 1-2 depicts the welcome page of \nthe Power BI website

Observations